In [1]:
import pyslha #For interacting with SLHA files
import subprocess #Running command
import scipy #Numeric Library
import sympy #symbolic computation - might not be used here
from scipy import optimize
from sympy import *
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
%matplotlib inline
from numpy import logspace, linspace 
from itertools import product
import glob #filehandling
from subprocess import call
import os
#Path to modified verison of Softsusy 
SoftSUSY_PATH = 'softsusy-4.1.4-Neil/softpoint.x' 
mHiggs = 125 #Higgs Mass for paper
print ("hello")

hello


In [2]:
import os
current_path = os.getcwd()
print(current_path)

/Users/glyph/SoftSusy/Shu_plotting


In [3]:
def slhaPoint(model='FGMCaseShu', mMess=1e12, tanBeta=10, epsu=2,epsd=2,epse=2, betaeps=.01, thetavev=0, 
              outputFile='test.slha', Lambda=1e6, mode='Point', LambdaGuess=6e5,particle=25):
    '''
    Code for interacting with SoftSUSY and running the models we've been looking at.
    The models themselves are encoded in a modified version of it. 
    This may seem like extra work because it accepts an arbitrary SLHA2 file, but since our models
    include fitted parameters, like the Yukawa couplings, we need to use the values that it calculates
    during the running. 
    The parameters:
    model - the name of the model in SoftSUSY
    mMess - the messenger scale
    tanBeta - tan beta
    beta2 - beta2
    beta3 - beta3
    outputFile - the name of the file to write to (this step could conceivably be removed by writing the console output to memory)
    Lambda - F/M
    mode - There are 3 modes
        'Fit' - uses Newton's method from a few different starting points
        'Bisect' - uses a Bisection routine from a few different brackets
        'Point' - runs a single point
    LambdaGuess - a guess at the value of Lambda for the 'Fit' mode.
    particle - the pdg code for the particle that we are interested in getting data for
    
    note that current implementation, fit and bisect only work for higgs, could fix, but it could only really work for
    particles with known values
    '''
    
    dataline ="./{} {} --mMess={} --LAMBDA={} --tanBeta={} --epsu={} --epsd={} --epse={} --thetavev={} --betaeps={} --cgrav=1. > {}"
    startingPoints = [LambdaGuess, 1e6, 5e6, 6e4]
    bracket, l = [(2,5), (5,10), (2,10), (2,8), (4,8), (1,3), (8,12)], 0
    
    #a helper functin that takes a value of x (Lambda), runs it (line 2) and then reads in the Higgs mass
    def f(x):
        string = dataline.format(SoftSUSY_PATH, model, str(mMess), str(x), 
                                 str(tanBeta), str(epsu), str(epsd), str(epse), str(thetavev), str(betaeps), outputFile)
        subprocess.Popen(string, stdout=None, shell=True).wait()
        print ("the string is ")
        print (string)
        return pyslha.read(outputFile).blocks['MASS'][particle]
    
    if mode == 'Fit':
        #print ('in fit')
        a, i = 0, 0
        while((a <= 0) & (i < len(startingPoints))):
            try:
                #Newton's method
                print ("in try")
                a = optimize.newton(f, startingPoints[i], tol=1e-4)
                print ("in newton")
                print (a)
            except:
                print ('got excepted')
                a = 0
                i += 1
        if (a>0):
            data = pyslha.read(outputFile)
            #accept or reject based on the deviation from the Higgs mass
            if (-0.1 < data.blocks['MASS'][25]-125 < 0.1) :
                try:
                    #if there's something in this location SoftSUSY has an error.
                    data.blocks['SPINFO'][3]
                except:
                    return data
    elif mode == 'Bisect':
        for k in bracket:
            print(k)
            try:
                l = optimize.brentq(f, k[0] * 1e5, k[1] * 1e5, rtol=1e-4)
                if l:
                    break
            except:
                #print('nada')
                continue
        if l:
            data = pyslha.read(outputFile)
            print(data.blocks['MASS'][25])
            if (-0.25 < data.blocks['MASS'][25]-125 < 0.25) :
                try:
                    data.blocks['SPINFO'][3]
                except:
                    return data
            else:
                #the bisection routine tends to get things close but not exact, so it calls 'Fit' if it's close.
                return slhaPoint(model, mMess, tanBeta, beta2, beta3,\
                              outputFile, Lambda, 'Fit', LambdaGuess=data.blocks['MINPAR'][1])
        else:
            #just to be safe (but is probably unneded)
            return slhaPoint(model, mMess, tanBeta, beta2, beta3,\
                              outputFile, Lambda, 'Fit')
    else:
        #runs a single point
        try:
            f(Lambda)
            data = pyslha.read(outputFile)
            return data.blocks['MASS'][particle], data
        except (pyslha.ParseError, RuntimeError):
            return "No Solution", 0

In [4]:
gd = slhaPoint(epse = .0001, epsd = .001, epsu = .0001)

the string is 
./softsusy-4.1.4-Neil/softpoint.x FGMCaseShu --mMess=1000000000000.0 --LAMBDA=1000000.0 --tanBeta=10 --epsu=0.0001 --epsd=0.001 --epse=0.0001 --thetavev=0 --betaeps=0.01 --cgrav=1. > test.slha


Ignoring unknown section type: in
Ignoring unknown section type: in


In [14]:
print(SPINFO is in gd[1].blocks)

SyntaxError: invalid syntax (<ipython-input-14-77277218fdd4>, line 1)

In [33]:
gd[1].blocks["SDFS"]

KeyError: ('S', 'D', 'F', 'S')

In [15]:
gd

(126.807235, <PySLHA Doc: 21 blocks, 34 decays, 0 xsections>)

In [17]:
gd

(126.807235, <PySLHA Doc: 21 blocks, 34 decays, 0 xsections>)